##Install Biopython

In [38]:
pip install biopython

##Import the necesarry packages

In [39]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqUtils import gc_fraction

##Read the Fasta file

In [40]:
record = SeqIO.read("sequence.fasta","fasta")
dna_seq = record.seq.upper()
print(f"Sequence_ID : {record.id}")
print(f"Sequence_length : {len(dna_seq)}bp")

Sequence_ID : NZ_CP014225.1
Sequence_length : 4659625bp


##Quality analysis of Unknown Sequence

GC Content

In [41]:
gc_cont = gc_fraction(dna_seq)*100
print(f"GC_Content : {gc_cont:.2f}%")

GC_Content : 50.78%


Checking DNA validation

In [42]:
valid_bases = set("ATGCN")
invalid_bases = set(dna_seq) - valid_bases
print(f"Total invalid bases : {invalid_bases}")

Total invalid bases : set()


Scientific interpretation:

“The DNA sequence is very large (~4.6 Mb), GC-balanced, and contains no invalid nucleotide characters, indicating a high-quality genomic DNA sequence suitable for downstream gene prediction analysis.”

##ORF

In [43]:
from Bio.Seq import Seq

seq = dna_seq
cleaned_seq_str = "".join([base if base in "ATGCN" else "N" for base in str(seq)])
cleaned_seq = Seq(cleaned_seq_str)

translated_frames = []

for strand, nuc in [("+", cleaned_seq), ("-", cleaned_seq.reverse_complement())]:
    for frame in range(3):
        protein = nuc[frame:].translate()
        translated_frames.append((strand, frame, protein))

##selecting the ORF with minimum aa length 300

In [44]:
orfs = []
min_aa_length = 300

for strand, frame, protein in translated_frames:
    for orf in protein.split("*"):
        if len(orf) >= min_aa_length:
            orfs.append((strand, frame, len(orf), orf))


In [45]:
print("Number of ORFs after filtering:", len(orfs))


Number of ORFs after filtering: 2289


In [46]:
orfs_sorted = sorted(orfs, key=lambda x: x[2], reverse=True)

for i, orf in enumerate(orfs_sorted[:5], start=1):
    print(f"ORF {i}: Length = {orf[2]} aa | Strand = {orf[0]} | Frame = {orf[1]}")


ORF 1: Length = 2387 aa | Strand = + | Frame = 0
ORF 2: Length = 1654 aa | Strand = - | Frame = 0
ORF 3: Length = 1570 aa | Strand = - | Frame = 1
ORF 4: Length = 1550 aa | Strand = - | Frame = 2
ORF 5: Length = 1545 aa | Strand = + | Frame = 0


##Filtering the Longest ORF

In [47]:
from Bio.Seq import Seq


orfs = []
min_aa_length = 300


for strand, frame, protein in translated_frames:
    for orf in protein.split("*"):
        if len(orf) >= min_aa_length:
            orfs.append((strand, frame, len(orf), orf))


if orfs:
    longest_orf = max(orfs, key=lambda x: x[2])

    strand, frame, aa_length, protein_seq = longest_orf

    print("Longest ORF details:")
    print("Protein length (aa):".ljust(20), aa_length)
    print("Strand:".ljust(20), strand)
    print("Frame:".ljust(20), frame)
else:
    print(f"No ORFs found with a minimum amino acid length of {min_aa_length} in any frame.")
    protein_seq = Seq("")

Longest ORF details:
Protein length (aa): 2387
Strand:              +
Frame:               0


In [48]:
print(protein_seq)

FSIILLSVFTFFRCARKGAFMLARSGKVSMATKKRSGEEINDRQILCGMGIKLRRLTAGICLITQLAFPMAAAAQGVVNAATQQPVPAQIAIANANTVPYTLGALESAQSVAERFGISVAELRKLNQFRTFARGFDNVRQGDELDVPAQVSEKKLTPPPGNSSDNLEQQIASTSQQIGSLLAEDMNSEQAANMARGWASSQASGAMTDWLSRFGTARITLGVDEDFSLKNSQFDFLHPWYETPDNLFFSQHTLHRTDERTQINNGLGWRHFTPTWMSGINFFFDHDLSRYHSRAGIGAEYWRDYLKLSSNGYLRLTNWRSAPELDNDYEARPANGWDVRAESWLPAWPHLGGKLVYEQYYGDEVALFDKDDRQSNPHAITAGLNYTPFPLMTFSAEQRQGKQGENDTRFAVDFTWQPGSAMQKQLDPNEVAARRSLAGSRYDLVDRNNNIVLEYRKKELVRLTLTDPVTGKSGEVKSLVSSLQTKYALKGYNVEATALEAAGGKVVTTGKDILVTLPAYRFTSTPETDNTWPIEVTAEDVKGNLSNREQSMVVVQAPTLSQKDSSVSLSTQTLNADSHSTATLTFIAHDAAGNPVVGLVLSTRHEGVQDITLSDWKDNGDGSYTQILTTGAMSGTLTLMPQLNGVDAAKAPAVVNIISVSSSRTHSSIKIDKDRYLSGNPIEVTVELRDENDKPVKEQKQQLNNAVSIDNVKPGVTTDWKETADGVYKATYTAYTKGSGLTAKLLMQNWNEDLHTAGFIIDANPQSAKIATLSASNNGVLANENAANTVSVNVADEGSNPINDHTVTFAVLSGSATSFNNQNTAKTDVNGLATFDLKSSKQEDNTVEVTLENGVKQTLIVSFVGDSSTAQVDLQKSKNEVVADGNDSVTMTATVRDAKGNLLNDVMVTFNVNSAEAKLSQTEVNSHDGIATATLTSLKNGDYRVTASVSSGSQANQQVNFIGDQSTAALTLSVPSGDITVTNTAPQYMTATLQDKNGNPL

“Since the input sequence represents a complete bacterial genome, sequence filtering was performed using ORF prediction. The genome was translated in all six reading frames using Biopython, and ORFs shorter than 300 amino acids were excluded. Among the remaining candidates, the longest ORF (2387 amino acids) encoded on the forward strand was selected for downstream analysis.”

##Converting the longest orf seq into a fastafile

In [49]:
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

orf_record = SeqRecord(
    protein_seq,
    id="Longest_ORF_2387aa",
    description="Predicted protein from ORF filtering (Strand +, Frame 0)"
)

SeqIO.write(orf_record, "orf_protein.fasta", "fasta")

print("ORF protein saved as orf_protein.fasta")


ORF protein saved as orf_protein.fasta


“Following ORF-based sequence filtering, the longest predicted ORF (2387 amino acids) was extracted and saved as a protein FASTA file for downstream homology-based comparison.”

#SEQUENCE COMPARISON (HOMOLOGY SEARCH)

PURPOSE OF THIS STEP (VERY IMPORTANT)

To identify whether the selected ORF is similar to any known genes/proteins and to infer its possible function based on evolutionary conservation.

This step answers:

“What is this gene similar to?”

“Does it already exist in other organisms?”

In [50]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO


In [51]:
record = SeqIO.read("orf_protein.fasta", "fasta")


In [52]:
result_handle = NCBIWWW.qblast(
    program="blastp",
    database="nr",
    sequence=record.seq
)


In [53]:
with open("blast_result.xml", "w") as out_handle:
    out_handle.write(result_handle.read())

print("BLAST results saved as blast_result.xml")


BLAST results saved as blast_result.xml


In [54]:
from Bio.Blast import NCBIXML

with open("blast_result.xml") as result_handle:
    blast_record = NCBIXML.read(result_handle)

top_alignment = blast_record.alignments[0]
top_hsp = top_alignment.hsps[0]

print("Top hit:", top_alignment.hit_def)
print("E-value:", top_hsp.expect)
print("Identity:", top_hsp.identities)


Top hit: adhesin [Escherichia coli B str. REL606] >gb|ACT43745.1| putative adhesin [Escherichia coli BL21(DE3)]
E-value: 0.0
Identity: 2379


“Homology-based similarity analysis was performed using BLASTp against the NCBI non-redundant protein database. The selected ORF showed a highly significant match (E-value = 0.0) to a putative adhesin protein from Escherichia coli, with near-complete sequence identity, indicating strong evolutionary conservation.”

##Converting interpro_results.tsv file into dataframe

In [56]:
import pandas as pd

interpro_df = pd.read_csv("interpro_results.tsv", sep="\t")

interpro_df.head()


Longest_ORF_2387aa  5a01f31c70fb3e07f33fb1e2d7729ffe  2387  Gene3D  \
0  Longest_ORF_2387aa  5a01f31c70fb3e07f33fb1e2d7729ffe  2387  Gene3D   
1  Longest_ORF_2387aa  5a01f31c70fb3e07f33fb1e2d7729ffe  2387  Gene3D   
2  Longest_ORF_2387aa  5a01f31c70fb3e07f33fb1e2d7729ffe  2387  Gene3D   
3  Longest_ORF_2387aa  5a01f31c70fb3e07f33fb1e2d7729ffe  2387  Gene3D   
4  Longest_ORF_2387aa  5a01f31c70fb3e07f33fb1e2d7729ffe  2387  Gene3D   

   G3DSA:2.60.40.10  Immunoglobulins  1883  1982  1.8E-13  T  29-01-2026  \
0  G3DSA:2.60.40.10  Immunoglobulins  1170  1269  4.0E-16  T  29-01-2026   
1  G3DSA:2.60.40.10  Immunoglobulins   865   962  2.5E-18  T  29-01-2026   
2  G3DSA:2.60.40.10  Immunoglobulins  1482  1584  2.7E-18  T  29-01-2026   
3  G3DSA:2.60.40.10  Immunoglobulins   763   864  1.9E-13  T  29-01-2026   
4  G3DSA:2.60.40.10  Immunoglobulins  1585  1687  1.2E-16  T  29-01-2026   

   IPR013783  Immunoglobulin-like fold  -  \
0  IPR013783  Immunoglobulin-like fold  -   
1  IPR013783  Immunoglobulin-like fold  -   
2  IPR013783  Immunoglobulin-like fold  -   
3  IPR013783  Immunoglobulin-like fold  -   
4  IPR013783  Immunoglobulin-like fold  -   

  Reactome:R-BTA-114608|Reactome:R-BTA-1236974|Reactome:R-BTA-1236977|Reactome:R-BTA-1257604|Reactome:R-BTA-1266695|Reactome:R-BTA-140834|Reactome:R-BTA-163125|Reactome:R-BTA-1632852|Reactome:R-BTA-1660661|Reactome:R-BTA-173736|Reactome:R-BTA-174577|Reactome:R-BTA-1971475|Reactome:R-BTA-198933|Reactome:R-BTA-2022870|Reactome:R-BTA-2022923|Reactome:R-BTA-2024101|Reactome:R-BTA-202424|Reactome:R-BTA-202427|Reactome:R-BTA-202430|Reactome:R-BTA-202433|Reactome:R-BTA-202733|Reactome:R-BTA-2029481|Reactome:R-BTA-2029482|Reactome:R-BTA-2029485|Reactome:R-BTA-210993|Reactome:R-BTA-216083|Reactome:R-BTA-2173791|Reactome:R-BTA-2424491|Reactome:R-BTA-3000178|Reactome:R-BTA-350054|Reactome:R-BTA-375276|Reactome:R-BTA-380972|Reactome:R-BTA-381426|Reactome:R-BTA-388844|Reactome:R-BTA-389948|Reactome:R-BTA-418594|Reactome:R-BTA-420029|Reactome:R-BTA-445355|Reactome:R-BTA-5173105|Reactome:R-BTA-5576892|Reactome:R-BTA-5627123|Reactome:R-BTA-5628897|Reactome:R-BTA-5673001|Reactome:R-BTA-5690714|Reactome:R-BTA-6785807|Reactome:R-BTA-6798695|Reactome:R-BTA-6804756|Reactome:R-BTA-6811558|Reactome:R-BTA-77387|Reactome:R-BTA-8849932|Reactome:R-BTA-8856825|Reactome:R-BTA-8856828|Reactome:R-BTA-8941856|Reactome:R-BTA-8957275|Reactome:R-BTA-8980692|Reactome:R-BTA-8983432|Reactome:R-BTA-8985947|Reactome:R-BTA-9013106|Reactome:R-BTA-9013404|Reactome:R-BTA-9013405|Reactome:R-BTA-9013408|Reactome:R-BTA-9020558|Reactome:R-BTA-9020958|Reactome:R-BTA-909733|Reactome:R-BTA-912526|Reactome:R-BTA-912694|Reactome:R-BTA-9609523|Reactome:R-BTA-977606|Reactome:R-BTA-983168|Reactome:R-BTA-983170|Reactome:R-BTA-983695|Reactome:R-BTA-9927354|Reactome:R-CEL-114608|Reactome:R-CEL-1257604|Reactome:R-CEL-1307965|Reactome:R-CEL-1474228|Reactome:R-CEL-177929|Reactome:R-CEL-190322|Reactome:R-CEL-190370|Reactome:R-CEL-190371|Reactome:R-CEL-190372|Reactome:R-CEL-190373|Reactome:R-CEL-190374|Reactome:R-CEL-190375|Reactome:R-CEL-190377|Reactome:R-CEL-1971475|Reactome:R-CEL-2022928|Reactome:R-CEL-2024096|Reactome:R-CEL-209968|Reactome:R-CEL-210991|Reactome:R-CEL-216083|Reactome:R-CEL-264876|Reactome:R-CEL-2682334|Reactome:R-CEL-3000157|Reactome:R-CEL-3000171|Reactome:R-CEL-3000178|Reactome:R-CEL-351906|Reactome:R-CEL-373752|Reactome:R-CEL-373753|Reactome:R-CEL-373756|Reactome:R-CEL-375165|Reactome:R-CEL-376176|Reactome:R-CEL-388844|Reactome:R-CEL-3928662|Reactome:R-CEL-3928663|Reactome:R-CEL-3928664|Reactome:R-CEL-3928665|Reactome:R-CEL-399954|Reactome:R-CEL-399955|Reactome:R-CEL-399956|Reactome:R-CEL-416482|Reactome:R-CEL-416550|Reactome:R-CEL-416572|Reactome:R-CEL-416700|Reactome:R-CEL-418885|Reactome:R-CEL-418886|Reactome:R-CEL-418889|Reactome:R-CEL-433692|Reactome:R-CEL-445144|Reactome:R-CEL-5140745|Reactome:R-CEL-5218920|Reactome:R-CEL-5620916|Reactome:R-CEL-5654219|Reactome:R-CEL-5654228|Reactome:R-CEL-5654688|Reactome:R-CEL-5654689|

“InterProScan analysis identified multiple conserved immunoglobulin-like domains (IPR013783) distributed throughout the protein sequence. These domains are characteristic of bacterial adhesins and are involved in protein–protein and cell–surface interactions. The presence of repeated Ig-like folds strongly supports the functional annotation of the protein as an adhesin.”

“The detection of multiple Ig-like domains suggests a modular architecture, which is typical of large surface-exposed bacterial adhesion proteins.”